<a href="https://colab.research.google.com/github/akankshakusf/Project-DeepLearning-SentimentAnalysis-DebertaHFT/blob/master/Sentiment_Analysis_with_BERT_in_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [1]:
!pip install transformers==4.37.2
!pip install keras==2.15.0
!pip install tensorflow==2.15.0

In [2]:
TF_USE_LEGACY_KERAS=1

In [ ]:
# !pip uninstall keras -y
# !pip uninstall tensorflow -y

Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0


In [3]:
import keras
print("Keras version:", keras.__version__)

import tensorflow as tf
print("TensorFlow version:", tf.__version__)
# Instead of tf.keras.__version__, use tf.__version__ to get the TensorFlow version,
# which includes Keras.
#Alternatively, if you need the standalone Keras version, use keras.__version__

import transformers
print("Transformers version:", transformers.__version__)

Keras version: 2.15.0
TensorFlow version: 2.15.0
Transformers version: 4.37.2


In [4]:
!pip install  datasets

# Imports

In [5]:
!pip install --upgrade tensorflow_text absl-py
!pip install tensorflow_text==2.15.0 --force-reinstall --no-deps
!pip install absl-py==1.4.0 --force-reinstall --no-deps

  Using cached tensorflow_text-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.0 kB)
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached tensorboard-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
  Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (21 kB)
Using cached tensorflow_text-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
Using cached absl_py-2.2.2-py3-none-any.whl (135 kB)
Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.9 MB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
Using cached ml_dtypes-0.5.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.7 MB)
Using cached tensorboard-2.19.0-py3-none-any.whl (5.5 MB)
  Attempt

  Using cached tensorflow_text-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.9 kB)
Using cached tensorflow_text-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.2 MB)
  Attempting uninstall: tensorflow_text
    Found existing installation: tensorflow-text 2.19.0
    Uninstalling tensorflow-text-2.19.0:
      Successfully uninstalled tensorflow-text-2.19.0
  Using cached absl_py-1.4.0-py3-none-any.whl.metadata (2.3 kB)
Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.2.2
    Uninstalling absl-py-2.2.2:
      Successfully uninstalled absl-py-2.2.2


In [7]:
#general package imports
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import cv2
import seaborn as sns
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
# import gensim.downloader as api
from PIL import Image

#tensorflow imports
import tensorflow as tf
import tensorflow_datasets as tfds
# import tensorflow_probability as tfp
from tensorflow import keras
from keras.models import Model
from keras.layers import Layer, Dense, Flatten,InputLayer, BatchNormalization,Dropout, Input, LayerNormalization
from keras.losses import BinaryCrossentropy, CategoricalCrossentropy, SparseCategoricalCrossentropy
from keras.metrics import Accuracy, TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer)
                          # ,RobertaTokenizerFast,
                          # DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,
                          # TFBertModel,create_optimizer)

RuntimeError: Failed to import transformers.models.bert.tokenization_bert_tf because of the following error (look up to see its traceback):
No module named 'keras.src.activations.activations'; 'keras.src.activations' is not a package

In [2]:
BATCH_SIZE=8

# Data Preparation for Bert Model

In [3]:
dataset_id = 'imdb'
dataset = load_dataset(dataset_id)

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
dataset['train'][0:5] #any five texts

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

## Bert Model

In [6]:
#import tokenizer
model_id = "bert-base-uncased"
#initialize tokenizer model
tokenizer = BertTokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
tokenizer.is_fast

True

In [8]:
#test the tokenizer
test_input_1='The weather of today is Great! zwp'
test_input_2='How are you doing?'
inputs = [test_input_1, test_input_2]
tokenizer.tokenize(inputs,)

['the',
 'weather',
 'of',
 'today',
 'is',
 'great',
 '!',
 'z',
 '##w',
 '##p',
 'how',
 'are',
 'you',
 'doing',
 '?']

In [9]:
#instantiate tokenizer
output=tokenizer(inputs, padding=True, truncation=True,max_length=128)
print(output)

{'input_ids': [[101, 1996, 4633, 1997, 2651, 2003, 2307, 999, 1062, 2860, 2361, 102], [101, 2129, 2024, 2017, 2725, 1029, 102, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]}


In [10]:
#pull 0th element of list
tokenizer.decode(output['input_ids'][0])

'[CLS] the weather of today is great! zwp [SEP]'

In [11]:
#pull 1st element of list
tokenizer.decode(output['input_ids'][1])

'[CLS] how are you doing? [SEP] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [12]:
def preprocess_function(examples):
  return tokenizer(examples["text"], padding=True,truncation=True,)

In [13]:
tokenized_dataset=dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [15]:
# tokenized_dataset['train'][0]

In [16]:
#data_collector = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [17]:
tf_train_dataset = tokenized_dataset['train'].to_tf_dataset(
      columns= ['input_ids','token_type_ids','attention_mask','label'],
      shuffle=True,
      batch_size=BATCH_SIZE,
)

In [18]:
tf_val_dataset = tokenized_dataset['test'].to_tf_dataset(
    columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    shuffle = True,
    batch_size=BATCH_SIZE,
)

In [19]:
for i in tf_train_dataset.take(1):
  print(i)

{'label': <tf.Tensor: shape=(8,), dtype=int64, numpy=array([0, 1, 0, 1, 1, 1, 1, 0])>, 'input_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[  101,  1045,  1005, ...,     0,     0,     0],
       [  101, 18396,  7078, ...,     0,     0,     0],
       [  101,  4735,  2135, ...,     0,     0,     0],
       ...,
       [  101,  2411,  7292, ...,  1026,  7987,   102],
       [  101, 28616,  8873, ...,     0,     0,     0],
       [  101,  2045,  2031, ...,     0,     0,     0]])>, 'token_type_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 

In [20]:
# we will push labels back as it is outputs
def swap_postions(dataset):
  return{'input_ids':dataset['input_ids'],
         'token_type_ids':dataset['token_type_ids'],
         'attention_mask':dataset['attention_mask'],},dataset['label']

In [21]:
tf_train_dataset = tf_train_dataset.map(swap_postions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset = tf_val_dataset.map(swap_postions).prefetch(tf.data.AUTOTUNE)

In [22]:
for i in tf_train_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[ 101, 1045, 6618, ..., 1005, 1055,  102],
       [ 101, 2009, 3138, ...,    0,    0,    0],
       [ 101, 1000, 1996, ...,    0,    0,    0],
       ...,
       [ 101, 1045, 2442, ...,    0,    0,    0],
       [ 101, 5506, 2932, ...,    0,    0,    0],
       [ 101, 2004, 6298, ...,    0,    0,    0]])>, 'token_type_ids': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(8, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>}, <tf.Tensor: shape=(8,), dtype=int64, numpy=array([0, 1, 1, 1, 1, 0, 0, 1])>

In [23]:
tf_train_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [24]:
tf_val_dataset

<_PrefetchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 512), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

# Modeling

## Based on TFBertForSequenceClassification

In [39]:
#Instantiate the model
model= TFBertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=1)
model.summary()

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  109482240 
                                                                 
 dropout_75 (Dropout)        multiple                  0 (unused)
                                                                 
 classifier (Dense)          multiple                  769       
                                                                 
Total params: 109483009 (417.64 MB)
Trainable params: 109483009 (417.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Custom BERT : Based on TFBertModel

In [26]:
# #### 2. Load model
# from transformers import TFBertModel
# model = TFBertModel.from_pretrained("bert-base-uncased", output_hidden_states = True)


model=TFBertModel.from_pretrained("bert-base-uncased")
model.summary()

AttributeError: 'NoneType' object has no attribute 'items'

In [89]:
model = TFBertModel.from_pretrained("bert-base-uncased")


input_ids=Input(shape = (512,),dtype=tf.int64,name='input_ids')
token_type_ids=Input(shape = (512,),dtype=tf.int64,name='token_type_ids')
attention_mask=Input(shape = (512,),dtype=tf.int64,name='attention_mask')

x = model([input_ids,token_type_ids,attention_mask])
print(x)
x=Dense(128,activation='relu')(x[0][:,0,:])
output=Dense(1,activation='sigmoid',name='label')(x)

custom_bert = tf.keras.Model(inputs=[input_ids,token_type_ids,attention_mask], outputs=output)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

ValueError: Exception encountered when calling layer 'tf_bert_model_15' (type TFBertModel).

Data of type <class 'keras.src.backend.common.keras_tensor.KerasTensor'> is not allowed only (<class 'tensorflow.python.framework.tensor.Tensor'>, <class 'bool'>, <class 'int'>, <class 'transformers.utils.generic.ModelOutput'>, <class 'tuple'>, <class 'list'>, <class 'dict'>, <class 'numpy.ndarray'>) is accepted for input_ids.

Call arguments received by layer 'tf_bert_model_15' (type TFBertModel):
  • input_ids=['<KerasTensor shape=(None, 512), dtype=int64, sparse=False, name=input_ids>', '<KerasTensor shape=(None, 512), dtype=int64, sparse=False, name=token_type_ids>', '<KerasTensor shape=(None, 512), dtype=int64, sparse=False, name=attention_mask>']
  • attention_mask=None
  • token_type_ids=None
  • position_ids=None
  • head_mask=None
  • inputs_embeds=None
  • encoder_hidden_states=None
  • encoder_attention_mask=None
  • past_key_values=None
  • use_cache=None
  • output_attentions=None


# Training

In [40]:
num_epochs = 3
batches_per_epoch = len(tokenized_dataset["train"])// BATCH_SIZE
total_train_steps = int(batches_per_epoch * num_epochs)

In [41]:
#import optimizer from hugging face
optimizer, schedule = create_optimizer(init_lr=2e-5,
                                       num_warmup_steps=0,
                                       num_train_steps=total_train_steps)

In [47]:
#compile the model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=Adam(2e-5),
    metrics=['accuracy'],
)

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.adam.Adam object at 0x7955b66e5810>

In [43]:
#fit the model
history = model.fit(
    tf_train_dataset.take(1000),
    validation_data = tf_val_dataset,
    epochs = 3,
)

Epoch 1/3


KeyboardInterrupt: 